In [ ]:
import numpy as np
# from scipy.optimize import curve_fit
import sys
import matplotlib.pyplot as plt
from matplotlib import gridspec

import iminuit
print("iminuit version:", iminuit.__version__)
from iminuit import Minuit
from iminuit.cost import LeastSquares
import inspect

In [ ]:
def curve_fit(f, x, y, sigma=None, p0=None, p0_lim=None):

    if sigma is None:
        sigma = np.ones_like(y)

    init_params = {}
    if not p0 is None:
        for i, n in enumerate(inspect.getfullargspec(f)[0][1:]):
            init_params[n]=p0[i]
    if not p0_lim is None:
        for i, n in enumerate(inspect.getfullargspec(f)[0][1:]):
            init_params["limit_"+n]=p0_lim[i]

    least_squares = LeastSquares(x, y, sigma, f)

    m = Minuit(least_squares, **init_params)

    m.migrad()    
    try:
        cov = np.array(m.matrix())
    except:
        try:
            m.hesse()
            cov = np.array(m.matrix())
        except:
            m.hesse()
            cov = np.array(m.matrix())
        

    return m.np_values(), cov



In [ ]:
def avg_stepsize_corr_b1(positions, nominal_positions, b1_strategy = "m", outliers=0, plot=False):

    nominal_step_size = nominal_positions[3] - nominal_positions[0]

    if b1_strategy == "c":
        pos_c   = positions[1::3]        
    elif b1_strategy == "m":
        pos_c = np.array( [ np.mean(positions[3*i:3*i+3]) for i in range(5)] ) 
    diffs =  np.diff( pos_c )

    if plot:
        plt.plot(range(len(diffs)), diffs, "o-b")

    if outliers == 0:
        avg_step_size = np.mean( diffs )
    else:
        excluded_indices = []

        diffs1  = diffs[:]
        for i in range(outliers):
            avg     = np.mean( diffs1 )
            i_extremum = np.argmax(np.abs(diffs1-avg))
            if np.abs(diffs1[i_extremum] - avg)<0.01:
                break
            excluded_indices.append( np.where(np.isclose(diffs, diffs1[i_extremum]))[0][0] )
            diffs1 = diffs1[ np.arange(len(diffs1)) != i_extremum ]
        avg = np.mean(diffs1)
        std = np.std(diffs1)
        lim = 3*std
        diffs2 = diffs[ np.abs( diffs-avg )<=lim ]
        avg_step_size = np.mean(diffs2)

        if plot:
            plt.plot(excluded_indices, diffs[np.array(excluded_indices)], "xr", markersize=12)
            plt.plot([-1,5], [avg, avg], "-", color="red")
            plt.fill_between([-1,5], [avg-lim, avg-lim], [avg+lim, avg+lim], color="red", alpha=0.2)
            plt.plot(np.arange(len(diffs))[ np.abs( diffs-avg )>lim ]+0.3, diffs[ np.abs( diffs-avg )>lim ], "*m", markersize=12)
    
    if plot:
        plt.plot([-1,5], [avg_step_size, avg_step_size], "--g")
        plt.show()

            
    od_estimates = diffs - avg_step_size
    new_stepsizes = od_estimates + nominal_step_size
    new_positions = np.concatenate(([0], np.cumsum(new_stepsizes)))
    # take the middle point as reference
    new_positions = new_positions - new_positions[2] + nominal_positions[7]

    return new_positions


In [ ]:
def avg_stepsize_corr_sep(positions, nominal_positions, steps_to_average=10, outliers=0, plot=False):

    nominal_step_size = nominal_positions[1] - nominal_positions[0]

    diffs = np.diff(positions)
    diffs = diffs[ (np.arange(len(diffs)) % 3) != 2]

    if plot:
        plt.plot(range(len(diffs)), diffs, "o-b")

    if steps_to_average == 10:
        if outliers == 0:
            avg_step_size = np.mean( diffs )
        else:
            # remove 4 outliers
            excluded_indices = []
            diffs1  =diffs[:]
            for i in range(outliers):
                avg  = np.mean( diffs1 )
                i_extremum = np.argmax(np.abs(diffs1-avg))
                if np.abs(diffs1[i_extremum] - avg)<0.01:
                    break
                excluded_indices.append( np.where(np.isclose(diffs, diffs1[i_extremum]))[0][0] )
                diffs1 = diffs1[ np.arange(len(diffs1)) != i_extremum ]
            avg = np.mean(diffs1)
            std = np.std(diffs1)
            lim = 3*std
            diffs2 = diffs[ np.abs( diffs-avg )<=lim ]
            avg_step_size = np.mean(diffs2)


            if plot:

                plt.plot(excluded_indices, diffs[np.array(excluded_indices)], "xr", markersize=12)
                plt.plot([-1,11], [avg, avg], "-", color="red")
                plt.fill_between([-1,11], [avg-lim, avg-lim], [avg+lim, avg+lim], color="red", alpha=0.2)
                plt.plot(np.arange(len(diffs))[ np.abs( diffs-avg )>lim ]+0.3, diffs[ np.abs( diffs-avg )>lim ], "*m", markersize=12)
                
        if plot:
            plt.plot([-1,11], [avg_step_size, avg_step_size], "--g")
            plt.show()

        od_estimates = diffs - avg_step_size
        new_stepsizes = od_estimates + nominal_step_size

    elif steps_to_average == 2:

        avg_step_sizes = np.repeat([ np.mean(diffs[2*i:2*i+2]) for i in range(5) ], 2)
        od_estimates = diffs - avg_step_sizes
        new_stepsizes = od_estimates + nominal_step_size

        if plot:
            for i in range(5):
                plt.plot(np.arange(len(avg_step_sizes))[2*i:2*i+2], avg_step_sizes[2*i:2*i+2], "--g")
            plt.show()

    # reference is always the middle point
    result = np.repeat(nominal_positions[1::3], 3)*1.0
    for i in range(len(result)):
        if i%3==0:
            result[i] -= new_stepsizes[i//3*2]
        elif i%3==2:
            result[i] += new_stepsizes[(i-2)//3*2+1]
    return result



In [ ]:
def get_headon_position(rates, rate_unc, separations, beamspot_positions ):

    def sg(x,a,m,s):
        return a*np.exp(-0.5*((x-m)/s)**2)
    
    a0 = np.max(rates)
    m0 = separations[1]
    s0 = np.abs(separations[1] - separations[0])
    popt_sg, pcov_sg = curve_fit(sg, separations, rates, sigma=rate_unc, 
                                    p0=[a0, m0, s0 ],
                                    p0_lim = [
                                        [a0/2, a0*2],
                                        [m0-s0, m0+s0],
                                        [s0/2, s0*2],
                                    ] )

    mu               = popt_sg[1]

    def lin(x,a,b):
        return a*x+b

    popt_lin, pcov_lin = curve_fit(lin, separations, beamspot_positions, sigma=[params["beamspot_position_uncertainty"]]*3, p0=[1, 0])
    a = popt_lin[0]
    b = popt_lin[1]

    # compute result uncertainty
    if np.allclose(pcov_lin, 0):
        std = a*pcov_sg[1,1]**0.5
    else:
        mu_randomized = mu + pcov_sg[1,1]**0.5*np.random.normal(size=1000)
        try:
            pcov_lin_sqrt = np.linalg.cholesky(pcov_lin)
        except Exception as e:
            print(e)
            print(pcov_lin)
            sys.exit()

        ab_randomized = popt_lin[:,np.newaxis] + np.einsum("ij,jk", pcov_lin_sqrt, np.random.normal(size=(2,1000)))
        std = np.std( ab_randomized[0,:]*mu_randomized + ab_randomized[1,:] )

    if std == 0:
        std = 1e-6

    return a*mu+b, std


In [ ]:
def lsc(params):

    ## nominal beampositions
    nominal_B1_positions = np.repeat((np.arange(5)*params["nominal_step_size_B1"]),3)
    scan_pattern = np.tile([-params["nominal_step_size_B2"], 0, params["nominal_step_size_B2"]], 5)
    nominal_B2_positions = nominal_B1_positions + scan_pattern

    ## OD and BPM
    if params["cumulative_OD"]:
        true_B1_positions = nominal_B1_positions * params["Nominal_LS_B1"] + np.cumsum(params["OD_B1"])
        true_B2_positions = nominal_B2_positions * params["Nominal_LS_B2"] + np.cumsum(params["OD_B2"])
    else:
        true_B1_positions = nominal_B1_positions * params["Nominal_LS_B1"] + params["OD_B1"]
        true_B2_positions = nominal_B2_positions * params["Nominal_LS_B2"] + params["OD_B2"]

    doros_B1_positions = true_B1_positions / params["Doros_LS_B1"] + np.random.normal(size=true_B1_positions.shape) * params["Doros_noise_sigma"] 
    doros_B2_positions = true_B2_positions / params["Doros_LS_B2"] + np.random.normal(size=true_B2_positions.shape) * params["Doros_noise_sigma"]

    arc_B1_positions = true_B1_positions / params["Arc_LS_B1"] + np.random.normal(size=true_B1_positions.shape) * params["Arc_noise_sigma"]
    arc_B2_positions = true_B2_positions / params["Arc_LS_B2"] + np.random.normal(size=true_B2_positions.shape) * params["Arc_noise_sigma"]

    avg_B1_positions = (doros_B1_positions + arc_B1_positions)/2
    avg_B2_positions = (doros_B2_positions + arc_B2_positions)/2

    ## OD correction of nominal

    if params["pos_strategy"]=="nominal, middle point":
        analysis_separations   = nominal_B1_positions - nominal_B2_positions
        analysis_B1_5positions = nominal_B1_positions[1::3]
    elif params["pos_strategy"]=="nominal, avg point":
        analysis_separations   = nominal_B1_positions - nominal_B2_positions
        analysis_B1_5positions = np.array([ np.mean(nominal_B1_positions[3*i:3*i+3]) for i in range(5)])
    elif params["pos_strategy"]=="doros, middle point":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_B1_5positions = doros_B1_positions[1::3]
    elif params["pos_strategy"]=="doros, avg point":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_B1_5positions = np.array([ np.mean(doros_B1_positions[3*i:3*i+3]) for i in range(5)])
    #######################
    elif params["pos_strategy"]=="doros, b1 stepsize, sep stepsize 10, avg point":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=0)
        analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "m", outliers=0)
    elif params["pos_strategy"]=="doros, b1 stepsize, sep stepsize 2, avg point":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=2, outliers=0)
        analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "m", outliers=0)
    elif params["pos_strategy"]=="doros, b1 stepsize, sep stepsize 10 outlier4, avg point":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=4)
        analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "m", outliers=0)
    elif params["pos_strategy"]=="doros, b1 stepsize outlier 1, sep stepsize 10, avg point":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=0)
        analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "m", outliers=1)
    elif params["pos_strategy"]=="doros, b1 stepsize outlier1, sep stepsize 10 outlier4, avg point":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=4)
        analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "m", outliers=1)
    #######################
    elif params["pos_strategy"]=="doros, b1 stepsize, sep stepsize 10, middle point":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=0)
        analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "c", outliers=0)
    elif params["pos_strategy"]=="doros, b1 stepsize, sep stepsize 2, middle point":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=2, outliers=0)
        analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "c", outliers=0)
    elif params["pos_strategy"]=="doros, b1 stepsize, sep stepsize 10 outlier4, middle point":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=4)
        analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "c", outliers=0)
    elif params["pos_strategy"]=="doros, b1 stepsize outlier 1, sep stepsize 10, middle point":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=0)
        analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "c", outliers=1)
    elif params["pos_strategy"]=="doros, b1 stepsize outlier1, sep stepsize 10 outlier4, middle point":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=4)
        analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "c", outliers=1)
    ###########################
    elif params["pos_strategy"]=="doros, b1 stepsize, avg point":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "c", outliers=1)
    elif params["pos_strategy"]=="doros, b1 stepsize outlier1, avg point":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "c", outliers=1)
    ###########################
    elif params["pos_strategy"]=="doros, b1 stepsize, middle point":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "c", outliers=1)
    elif params["pos_strategy"]=="doros, b1 stepsize outlier1, middle point":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "c", outliers=1)


    ## beamspot location 
    true_beamspot_positions = ( 
            true_B1_positions/params["Sigma_B1"]**2 + 
            true_B2_positions/params["Sigma_B2"]**2
        )/(params["Sigma_B1"]**-2+params["Sigma_B2"]**-2)
        
    ## rates
    true_rates       = np.exp(-0.5 * (true_B1_positions-true_B2_positions)**2 / (params["Sigma_B1"]**2+params["Sigma_B2"]**2) )
    true_rates_unc   = true_rates**0.5 * np.max(true_rates)**0.5 * params["rate_runcertainty_at_head_on"]

    if params["randomize_rates"]:
        randomized_rates = true_rates + np.random.normal(size=true_rates_unc.shape) * true_rates_unc
    else:
        randomized_rates = true_rates

    ## calculate head-on beamspot position
    
    computed_head_on_bs_position = []
    computed_head_on_bs_position_unc = []
    for i in range(5):

        v, u = get_headon_position(
            randomized_rates[3*i:3*i+3],
            true_rates_unc[3*i:3*i+3],
            analysis_separations[3*i:3*i+3],
            true_beamspot_positions[3*i:3*i+3])

        if not (np.isfinite(v) and np.isfinite(u)):
            print(i)
            print(u, v)
            print([
                randomized_rates[3*i:3*i+3],
                true_rates_unc[3*i:3*i+3],
                analysis_separations[3*i:3*i+3],
                true_beamspot_positions[3*i:3*i+3]
            ])
            print(true_B1_positions-true_B2_positions)
            print(analysis_separations)
            sys.exit()

        computed_head_on_bs_position.append(v) 
        computed_head_on_bs_position_unc.append(u)


    ## fit linear function
    def lin(x,a,b):
        return a*x+b

    a0 = (computed_head_on_bs_position_unc[-1]-computed_head_on_bs_position_unc[0])/(analysis_B1_5positions[-1]-analysis_B1_5positions[0])
    b0 = 0
    popt_lin, pcov_lin = curve_fit(lin, analysis_B1_5positions, computed_head_on_bs_position, sigma=computed_head_on_bs_position_unc, p0=[a0, b0])
    a = popt_lin[0]
    b = popt_lin[1]

    if not (np.isfinite(a) and np.isfinite(b)):
        print(a, b)
        print([
            analysis_B1_5positions,
            computed_head_on_bs_position,
        ])
        sys.exit()

    return a, pcov_lin[0,0]**0.5

In [ ]:
N = 1000
sigma_OD = 0

params = {
    "Nominal_LS_B1":0.995,
    "Nominal_LS_B2":0.99,
    "Doros_LS_B1":1.005,
    "Doros_LS_B2":1.01,
    "Arc_LS_B1":1.005,
    "Arc_LS_B2":1.01,
    "Sigma_B1":90, #micron
    "Sigma_B2":90, #micron
    "nominal_step_size_B1": 130, #micron
    "nominal_step_size_B2": 130, #micron
    "Doros_noise_sigma":1, # micron
    "Arc_noise_sigma":0, # micron
    "rate_runcertainty_at_head_on":0.005, #relative
    "randomize_rates":True,
    "beamspot_position_uncertainty":1, #micron
}

for sigma_OD in [0, 5, 10]:
    for cumulative_OD in [False, True]:
        for pos_strategy in [
                                # "nominal, middle point",
                                # "nominal, avg point",
                                # #
                                # "doros, middle point",
                                # "doros, avg point",
                                # #
                                # "doros, b1 stepsize, sep stepsize 10, avg point",
                                # "doros, b1 stepsize, sep stepsize 2, avg point",
                                # "doros, b1 stepsize, sep stepsize 10 outlier4, avg point",
                                # "doros, b1 stepsize outlier 1, sep stepsize 10, avg point",
                                # "doros, b1 stepsize outlier1, sep stepsize 10 outlier4, avg point",
                                # #
                                # "doros, b1 stepsize, sep stepsize 10, middle point",
                                # "doros, b1 stepsize, sep stepsize 2, middle point",
                                # "doros, b1 stepsize, sep stepsize 10 outlier4, middle point",
                                # "doros, b1 stepsize outlier 1, sep stepsize 10, middle point",
                                # "doros, b1 stepsize outlier1, sep stepsize 10 outlier4, middle point",
                                # #
                                "doros, b1 stepsize, avg point",
                                "doros, b1 stepsize outlier1, avg point",
                                #
                                "doros, b1 stepsize, middle point",
                                "doros, b1 stepsize outlier1, middle point",
                            ]:

            settings_txt = "\n".join([
                "Settings:",
                "pos_strategy:\n  " + "\n  ".join(pos_strategy.split(',')),
                "Nominal_LS_B1: {}".format(params["Nominal_LS_B1"]),
                "Nominal_LS_B2: {}".format(params["Nominal_LS_B2"]),
                "Doros_LS_B1: {}".format(params["Doros_LS_B1"]),
                "Doros_LS_B2: {}".format(params["Doros_LS_B2"]),
                # "Arc_LS_B1: {}".format(params["Arc_LS_B1"]),
                # "Arc_LS_B2: {}".format(params["Arc_LS_B2"]),
                "Sigma_B1: {} um".format(params["Sigma_B1"]),
                "Sigma_B2: {} um".format(params["Sigma_B2"]),
                "nominal_step_size_B1: {} um".format(params["nominal_step_size_B1"]),
                "nominal_step_size_B2: {} um".format(params["nominal_step_size_B2"]),
                "cumulative_OD: {}".format(cumulative_OD),
                "sigma_OD: {} um".format(sigma_OD),
                "Doros_noise_sigma: {} um".format(params["Doros_noise_sigma"]),
                "rate_runcertainty_at_head_on: {}%".format(params["rate_runcertainty_at_head_on"]*100),
                "randomize_rates: {}".format(params["randomize_rates"]),
                "beamspot_position_uncertainty: {}".format(params["beamspot_position_uncertainty"]),
            ])
            

            ls_results = []
            ls_result_errs = []
            for i in range(N):

                params.update(
                    {
                        "cumulative_OD":cumulative_OD,
                        "OD_B1":np.random.normal(size=15)*sigma_OD,
                        "OD_B2":np.random.normal(size=15)*sigma_OD,
                        "pos_strategy":pos_strategy,
                    })

                # try:
                ls, lse = lsc(params)
                ls_results.append(ls)
                ls_result_errs.append(lse)
                # except RuntimeError as e:
                #     print(e)


            fig = plt.figure(figsize=(10,10))
            fig.patch.set_facecolor('white')
            gs = gridspec.GridSpec(2,2)
            # gs.update(hspace=0.1)

            ax0  = fig.add_subplot(gs[0,0])
            ax0.set_xlabel("LS", fontsize=16)
            ax1 = fig.add_subplot(gs[1,0])
            ax1.set_xlabel("LS unc", fontsize=16)
            for ax in [ax0, ax1]:
                ax.grid(True)
                ax.tick_params(axis='y', which='major', direction="in", labelsize=16, pad = 8)
                ax.tick_params(axis='x', which='major', direction="in", labelsize=16, pad = 12)
                for axis in ['top','bottom','left','right']:
                    ax.spines[axis].set_linewidth(0.5)

            ax2 = fig.add_subplot(gs[:,1])   
            for axis in ['top','bottom','left','right']:
                ax2.spines[axis].set_linewidth(0.0)
            ax2.tick_params(
                    axis='both',          
                    which='both',     
                    bottom=False,      
                    top=False, 
                    left=False,      
                    right=False,         
                    labelbottom=False,
                    labelleft=False)

            

            ax0.hist(ls_results, 50, density=False, facecolor='b', alpha=0.75)
            ax1.hist(ls_result_errs, 50, density=False, facecolor='g', alpha=0.75)
            # ax0.hist((np.array(ls_results)-1)*100, 50, density=False, facecolor='b', alpha=0.75)
            # ax1.hist((np.array(ls_results)-1)*1000, 50, density=False, facecolor='g', alpha=0.75)


            ax0.set_ylim(0, ax0.get_ylim()[1]*1.15)
            ax0.text(0.02, 0.97, "Mean: {:0.4f}$\\pm${:0.4f}\nStd:{:0.4f}".format(np.mean(ls_results), np.std(ls_results)/N**0.5, np.std(ls_results)),
                            horizontalalignment='left',
                            verticalalignment='top',
                            transform=ax0.transAxes,
                            fontname='sans-serif',
                            fontweight='bold',
                            fontsize=14)

            ax1.set_ylim(0, ax1.get_ylim()[1]*1.15)
            ax1.text(0.02, 0.97, "Mean: {:0.4f}$\\pm${:0.4f}\nStd:{:0.4f}".format(np.mean(ls_result_errs), np.std(ls_result_errs)/N**0.5,  np.std(ls_result_errs)),
                            horizontalalignment='left',
                            verticalalignment='top',
                            transform=ax1.transAxes,
                            fontname='sans-serif',
                            fontweight='bold',
                            fontsize=14)

            ax2.text(0.02, 0.97, settings_txt,
                            horizontalalignment='left',
                            verticalalignment='top',
                            transform=ax2.transAxes,
                            fontname='sans-serif',
                            fontweight='normal',
                            fontsize=14)

            fig.savefig("sigma_OD_{}_cumulative_{}_strategy_{}.png".format(sigma_OD, cumulative_OD, pos_strategy))
            
